In [4]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_stock(ticker):
    return yfinance.Ticker(ticker)


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = get_stock(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = get_stock(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = get_stock(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [7]:
get_ticker({"company_name": "Cloudflare"})

'Ticker Symbol NET Full Company Profile Financial Performance In 2022, Cloudflare\'s revenue was $975.24 million, an increase of 48.57% compared to the previous year\'s $656.43 million. Losses were -$193.38 million, -25.71% less than in 2021. Financial Statements Analyst Forecast According to 22 analysts, the average rating for NET stock is "Hold." Malik Ahmed Khan Nov 3, 2023 We are maintaining our $60 fair value estimate for narrow-moat Cloudflare after the firm reported strong third-quarter results and guidance mostly in line with our... Cloudflare, Inc. is a global cloud services provider that delivers a range of services to businesses of all sizes and in all geographies. The Company\'s network serves as a scalable, unified control plane to deliver security, performance, and reliability across on-premise, hybrid, cloud, and software-as-a-service (SaaS) applications. Cloudflare Stock Price, News & Analysis (NYSE:NET) $74.25 +0.85 (+1.16%) (As of 11:15 AM ET) Compare Today\'s Range $

In [8]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )
assistant_id = "asst_2Yrj0JjLevOUeOxnjleCcYiR"

In [11]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is good to buy",
        },
    ]
)

In [12]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

In [22]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(run_id=run_id, thread_id=thread_id)


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id,
    )
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}:{message.content[0].text.value}")

In [23]:
get_messages(thread_id=thread.id)

user:Text(annotations=[], value='I want to know if the Salesforce stock is good to buy')
assistant:Text(annotations=[], value="To assess whether Salesforce (CRM) stock is a good buy, we would need to consider various factors which typically include looking at the company's financial health, stock performance, and current market conditions. Here are some steps we can follow:\n\n1. **Financial Health**: We can analyze Salesforce's financial documents, such as the income statement and balance sheet.\n2. **Stock Performance**: We can also look at the historical performance of Salesforce's stock over the last 100 days to gauge how the stock has been trending.\n3. **Latest News and Analysis**: It's also important to look at any recent news about Salesforce that might affect stock performance.\n4. **Market Conditions**: We need to consider the overall market conditions and any sector-specific factors that might impact the stock.\n5. **Valuation and Ratios**: Examining key ratios such as the P